
# Otsu Method 

In this method we detect the edge's of the image 

In [11]:
from __future__ import absolute_import 
from __future__ import division
from __future__ import print_function

In [12]:
import cv2
import pylab as plt
import numpy as np

In [13]:
def varienceCalculate(average, histgram):
    vrience = 0
    for i in range(len(histgram)):
        varience += (histgram[i] - average) ** 2
    varience /= len(histgram)
    return variance

In [2]:
import numpy as np
a = np.random.rand(12,2)
a

array([[0.8849601 , 0.95143583],
       [0.66930872, 0.53470112],
       [0.07457593, 0.17111031],
       [0.71012886, 0.83723317],
       [0.06955176, 0.48981681],
       [0.77557906, 0.01588467],
       [0.169802  , 0.17057183],
       [0.05091569, 0.13757776],
       [0.39084971, 0.48733396],
       [0.29387811, 0.17515667],
       [0.36457179, 0.39511912],
       [0.13044492, 0.96868589]])

In [15]:
a/=len(a)
a

array([[0.0638758 , 0.04047314],
       [0.08319931, 0.07233665],
       [0.03146999, 0.05385992],
       [0.0711082 , 0.01559239],
       [0.0327321 , 0.03210233],
       [0.07521494, 0.07254533],
       [0.05296038, 0.01805167],
       [0.07064814, 0.06108527],
       [0.0663469 , 0.01037247],
       [0.05507742, 0.06767994],
       [0.0113735 , 0.01970784],
       [0.0139283 , 0.07147995]])

In [16]:
a = 12
a/= a
a

1.0

In [17]:
def averageAndpixelSumCalculate(histgram):
    average = pixelSum = 0
    for i in range(len(histgram)):
        pixelSum += histgram[i]
        brightnessValue = histgram[i] * i
    average = brightnessValue / len(histgram)
    return pixelSum, average

In [18]:
def within_betweenCV(pixelSum1, average1, variance1, pixelSum2, average2, variance2):
    betweenClassVariance = (pixelSum1 * pixelSum2 * ((average1 - average2)**2)/((pixelSum1 + pixelSum2)**2))
    withinClassVariance = (pixelSum1 * varience1 + pixelSum2 * varience2) /(pixelSum1 + pixelSum2)
    return betweenClassVariance, withClassVariance

In [19]:
def calculateAll(blacList, whiteList):
    b_size, b_average = averageAndpixelSumCalculate(blackList)
    w_size, w_average = averageAndpixelSumCalculate(whiteList)
    
    b_variance = varianceCalculate(b_average, blacklist)
    w_variance = varianceCalculate(w_average, whiteList)
    
    betweenCV, withinCV = within_beweenCV(b_size, b_average, b_variance, w_size, w_variance)
    
    totalVariance = betweenCV + withinCV
    seperationMetrics = betweenCV / (totalVariance - betweenCV)
    return separationMetrics

In [20]:
def main():
    image_path = "D:/AI_for_health/chest_X_rays/images_001/images/00000013_036.png"
    image = cv2.imread(image_path,0)
    histgram = cv2.calHist([image], [0], None, [256], [0,256])
    size = 256
    listSM = [0 for i in range(size)]
    for i in range(size):
        if i != 0 and i != size-1:
            blackList = histgram[0:i]
            whiteList = histgram[i:size]
            listSM[i] = calculateAll(blackList, whiteList)
        elif i == 0 or  i == size-1:
            listSM[i] = 0
    maxValue = 0
    for i in range(size):
        if listSM[i] > maxValue:
            maxValue = listSM[i]
            maxValueIndex = i
        print("Treshold", maxValueIndex, "Desu")
        
        output_otsu = np.zeros((len(image), len(image[0])))
        for i in range(len(image)):
            for j in range(len(image[0])):
                if image[i][j] > maxValueIndex:
                    output_otsu[i][j] = 225
                else:
                    output_otsu[i][j] = 0
                    
    average_histgram = int(len(histgram)/2)
    output_average = image.copy()
    output_average[output_average >= average_histgram] = 255
    output_average[output_average < average_histgram] = 0
    # Adaptive gaussian Threshold
    aGH = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 89, 7)
    
    cv2.imwrite("gray.jpj", image)
    cv2.imwrite("average.jpg", output_average)
    cv2.imwrite("otsu.jpg", output_otsu)
    cv2.imwrite("Adaptiv_Gaussian_Thresholding.jpg", aGH)
    
    cv2.imshow("input", image)
    cv2.imshow("average", output_average)
    cv2.imshow("otsu", output_otsu)
    cv2.imshow("adaptive gaussian", aGH)
    
    plt.plot(histgram)                                      
    plt.axvline(x=maxValueIndex, color='red', label='otsu') 
    plt.axvline(x=average_histgram, color='green', label='average')
    plt.legend(loc='upper right')                           
    plt.title("histgram of brightness")                     
    plt.xlabel("brightness")                                
    plt.ylabel("frequency")                                 
    plt.xlim([0, 256])                                      
    plt.show()
    
    if __name__ == '__main__':
        main()